In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 선형회귀 - simple model

In [16]:
X_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([2,4,6]).view(size=(-1,1))

# requires_grad 업데이트 될 때마다 값이 변경되는 함수
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

h = X_train * W + b
cost = torch.mean((h - y_train) ** 2)
cost

tensor(18.6667, grad_fn=<MeanBackward0>)

In [ ]:
optm = optim.SGD([W, b], lr=0.01)

# 미니배치마다 기울기를 0으로 초기화
# 안하면 누적합을 해버려서 이상한 값이 됨 (RNN 계산용)
optm.zero_grad()
cost.backward() # 기울기 계산
optm.step() # 파라미터 업데이트

In [17]:
optm = optim.SGD([W, b], lr=0.01)
epochs = 2000

for i in range(epochs):
    h = X_train * W + b
    cost = torch.mean((h - y_train)**2)
    
    optm.zero_grad()
    cost.backward() # 기울기 계산
    optm.step()
    
    if i % 200 == 0:
        print(W.item(), b.item(), cost.item())

0.18666668236255646 0.07999999821186066 18.66666603088379
1.80009925365448 0.4544214904308319 0.029766537249088287
1.876473307609558 0.28080499172210693 0.011366334743797779
1.9236682653427124 0.1735200434923172 0.00434019835665822
1.952831506729126 0.10722480714321136 0.0016572937602177262
1.9708527326583862 0.06625857204198837 0.0006328357267193496
1.9819889068603516 0.040943603962659836 0.0002416476490907371
1.9888702630996704 0.025300635024905205 9.227206464856863e-05
1.9931223392486572 0.015634294599294662 3.523503983160481e-05
1.995750069618225 0.009661080315709114 1.3454421605274547e-05


In [36]:
torch.manual_seed(42)
torch.empty(4,2).uniform_(0, 1)

tensor([[0.8823, 0.9150],
        [0.3829, 0.9593],
        [0.3904, 0.6009],
        [0.2566, 0.7936]])

## 선형회귀 - 행렬 연산

In [74]:
X_train = torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]]) 
y_train = torch.FloatTensor([152, 185, 180, 196, 142])[..., None]

W = torch.zeros((X_train.shape[1], 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optm = optim.SGD([W, b], lr=1e-5)

In [75]:
for i in range(21):
    y_hat = X_train.matmul(W) + b # X @ W
    loss = torch.mean((y_hat - y_train)**2)
    
    optm.zero_grad()
    loss.backward() # 기울기 계산
    optm.step()
    
    print(y_hat.squeeze().detach(), loss.item())
    

tensor([0., 0., 0., 0., 0.]) 29661.80078125
tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) 9537.6943359375
tensor([104.5421, 125.6208, 119.2478, 134.7861,  95.8280]) 3069.5908203125
tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) 990.6702880859375
tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) 322.48187255859375
tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) 107.7170639038086
tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) 38.687496185302734
tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) 16.499042510986328
tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) 9.365655899047852
tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625]) 7.071113586425781
tensor([153.5285, 184.4835, 175.1338, 197.9415, 140.7325]) 6.331847190856934
tensor([153.7572, 184.7582, 175.3958, 198.2360, 140.9424]) 6.092532157897949
tensor([153.8868, 184.9138, 175.5449, 198.4026, 141.0613]) 6.013816833496094
tensor([153.9602, 185.0019, 175.6299, 

## layer와 내장 mse - simple linear

In [82]:
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

X_train = torch.FloatTensor([1,2,3])[..., None]
y_train = torch.FloatTensor([2,4,6])[..., None]

model = nn.Linear(1, 1)


# 랜덤 초기화 W, b
# requires_grad = True
list(model.parameters())

[Parameter containing:
 tensor([[0.5153]], requires_grad=True), Parameter containing:
 tensor([-0.4414], requires_grad=True)]

In [83]:
sgd = torch.optim.SGD(model.parameters(), lr=1e-2)

for epoch in range(1, 1001):
    pred = model(X_train)
    loss = F.mse_loss(pred, y_train)
    
    sgd.zero_grad()
    loss.backward()
    sgd.step()
    
    if epoch % 100 == 0:
        print(f'{loss.item():.6f}')

0.002804
0.001733
0.001071
0.000662
0.000409
0.000253
0.000156
0.000096
0.000060
0.000037


In [87]:
X_test = torch.FloatTensor([[4]])
y_hat = model(X_test)
print(y_hat.item())

7.987856864929199


## layer와 내장 mse - multi linear

In [103]:
torch.manual_seed(1)
X_train = torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]]) 
y_train = torch.FloatTensor([152, 185, 180, 196, 142])[..., None]

model = nn.Linear(X_train.shape[1], y_train.shape[1])

sgd = torch.optim.SGD(model.parameters(), lr=1e-5)

for epoch in range(2000):
    pred = model(X_train)
    cost = F.mse_loss(pred, y_train)
    
    sgd.zero_grad()
    cost.backward()
    sgd.step()
    
    if epoch % 100 == 0:
        print(cost.item())

31667.59765625
0.2259877622127533
0.22390982508659363
0.22192999720573425
0.22005856037139893
0.21826967597007751
0.2165713608264923
0.21495532989501953
0.21341314911842346
0.2119491845369339
0.2105575054883957
0.20923714339733124
0.20797093212604523
0.20676425099372864
0.2056158035993576
0.20452742278575897
0.20347893238067627
0.2024870365858078
0.20154185593128204
0.2006380259990692


In [104]:
X_test = torch.FloatTensor([[73, 80, 75]])
model(X_test)

tensor([[151.2305]], grad_fn=<AddmmBackward>)

## 클래스 구현

In [115]:
torch.manual_seed(1)

class myLinear(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

model = myLinear()

X_train = torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]]) 
y_train = torch.FloatTensor([152, 185, 180, 196, 142])[..., None]

sgd = torch.optim.SGD(model.parameters(), lr=1e-5)

In [116]:
for epoch in range(2000):
    pred = model(X_train)
    cost = F.mse_loss(pred, y_train)
    
    sgd.zero_grad()
    cost.backward()
    sgd.step()
    
    if epoch % 200 == 0:
        print(cost.item())

31667.59765625
0.22390982508659363
0.22005856037139893
0.2165713608264923
0.21341314911842346
0.2105575054883957
0.20797093212604523
0.2056158035993576
0.20347893238067627
0.20154185593128204


## 배치적용

In [126]:
from torch.utils.data import DataLoader, TensorDataset

X_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

ds = TensorDataset(X_train, y_train)
list(ds)

[(tensor([73., 80., 75.]), tensor([152.])),
 (tensor([93., 88., 93.]), tensor([185.])),
 (tensor([89., 91., 90.]), tensor([180.])),
 (tensor([ 96.,  98., 100.]), tensor([196.])),
 (tensor([73., 66., 70.]), tensor([142.]))]

In [125]:
# import tensorflow as tf

# X_train = tf.constant([[73,  80,  75], 
#                        [93,  88,  93], 
#                        [89,  91,  90], 
#                        [96,  98,  100],   
#                        [73,  66,  70]])
# y_train = tf.constant([[152],  [185],  [180],  [196],  [142]])

# ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# list(ds)

[(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([73, 80, 75], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([152], dtype=int32)>),
 (<tf.Tensor: shape=(3,), dtype=int32, numpy=array([93, 88, 93], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([185], dtype=int32)>),
 (<tf.Tensor: shape=(3,), dtype=int32, numpy=array([89, 91, 90], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([180], dtype=int32)>),
 (<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 96,  98, 100], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([196], dtype=int32)>),
 (<tf.Tensor: shape=(3,), dtype=int32, numpy=array([73, 66, 70], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([142], dtype=int32)>)]

In [137]:
torch.manual_seed(1)
dataloader = DataLoader(ds, batch_size=2, shuffle=True)
for idx, sample in enumerate(dataloader):
    print(idx, sample)


0 [tensor([[ 96.,  98., 100.],
        [ 89.,  91.,  90.]]), tensor([[196.],
        [180.]])]
1 [tensor([[73., 66., 70.],
        [93., 88., 93.]]), tensor([[142.],
        [185.]])]
2 [tensor([[73., 80., 75.]]), tensor([[152.]])]


In [139]:
model = nn.Linear(X_train.shape[1], y_train.shape[1])
optm = torch.optim.SGD(model.parameters(), lr=1e-5)

for epoch in range(2000):
    for batch_idx, sample in enumerate(dataloader):
        X_train, y_train = sample
        
        pred = model(X_train)
        cost = F.mse_loss(pred, y_train)
        
        optm.zero_grad()
        cost.backward()
        optm.step()
        
        if epoch % 200 == 0:
            print('Batch : {}/{}, Loss : {}'.format(batch_idx+1, len(dataloader), cost.item()))

Batch : 1/3, Loss : 23353.75390625
Batch : 2/3, Loss : 8360.4375
Batch : 3/3, Loss : 3192.3310546875
Batch : 1/3, Loss : 0.2322549670934677
Batch : 2/3, Loss : 1.363370656967163
Batch : 3/3, Loss : 0.7043383121490479
Batch : 1/3, Loss : 0.30122122168540955
Batch : 2/3, Loss : 1.9600763320922852
Batch : 3/3, Loss : 0.03895587846636772
Batch : 1/3, Loss : 1.245780348777771
Batch : 2/3, Loss : 0.3653629422187805
Batch : 3/3, Loss : 0.07200692594051361
Batch : 1/3, Loss : 1.0821043252944946
Batch : 2/3, Loss : 0.12899114191532135
Batch : 3/3, Loss : 0.7164531350135803
Batch : 1/3, Loss : 0.7354222536087036
Batch : 2/3, Loss : 0.448956161737442
Batch : 3/3, Loss : 0.5318027138710022
Batch : 1/3, Loss : 0.14216792583465576
Batch : 2/3, Loss : 1.0472912788391113
Batch : 3/3, Loss : 0.9741981625556946
Batch : 1/3, Loss : 0.2624499797821045
Batch : 2/3, Loss : 0.39898037910461426
Batch : 3/3, Loss : 1.4127304553985596
Batch : 1/3, Loss : 0.367527037858963
Batch : 2/3, Loss : 0.509928822517395
B

In [140]:
X_test = torch.FloatTensor([[73,80,75]])
model(X_test)

tensor([[151.4232]], grad_fn=<AddmmBackward>)